In [2]:
from Bio.PDB import PDBParser
from Bio.PDB.Selection import unfold_entities
import numpy as np
from Bio.SeqUtils import seq1
import textwrap

In [3]:
def insert_gap(aaindex, residue_list):
    """
    Inserts a gap in the residue list in the position corresponding to the amino acid index.
    """
    tmplist = residue_list.copy()
    for index, res in enumerate(residue_list):
        if aaindex < res[0]:  # Given index less than current index in list
            tmplist.insert(index, (aaindex, "-"))
            return tmplist
    tmplist.append((aaindex, "-"))
    return tmplist

In [4]:
def insert_gap(aaindex, residue_list):
    """
    Inserts a gap in the residue list in the position corresponding to the amino acid index.
    """
    tmplist = residue_list.copy()
    for index, res in enumerate(tmplist):
        if aaindex < res[0]:  # Given index less than current index in list
            residue_list.insert(index, (aaindex, "-"))
            return tmplist
    residue_list.append((aaindex, "-"))

In [5]:
testseq = [(100,"A"), (101,"B"), (105,"C")]

In [6]:
insert_gap(106, insert_gap(103,insert_gap(90,insert_gap(92,insert_gap(104,testseq)))))

In [7]:
testseq

[(100, 'A'), (101, 'B'), (104, '-'), (105, 'C')]

In [8]:
insert_gap(103, testseq)

[(100, 'A'), (101, 'B'), (104, '-'), (105, 'C')]

In [9]:
testseq

[(100, 'A'), (101, 'B'), (103, '-'), (104, '-'), (105, 'C')]

In [5]:
# Read structure and extract present and missing residues
pdbparser = PDBParser()
structure = pdbparser.get_structure('dbl6e', '/home/ijpulidos/workdir/uniandes/MPTG-CPB/Thesis/PDBs/2y8d.pdb')
residues = unfold_entities(structure, "R")
missing_residues = structure.header['missing_residues']

In [7]:
residues_list = [(residue.id[1], seq1(residue.resname)) for residue in residues if residue.id[0]==" " ]
for mis_res in missing_residues:
    insert_gap(mis_res['ssseq'], residues_list)

In [8]:
cadena = "".join(np.array(residues_list)[:,1])

In [ ]:
textwrap.wrap(cadena, width=75, break_on_hyphens=False)

In [ ]:
full_seq.replace?

In [9]:
# Building full sequence
full_seq = cadena
for mis_res in missing_residues:
    full_seq = full_seq.replace("-", seq1(mis_res["res_name"]), 1)

In [10]:
full_seq

'GNDYICNKYKNIHDRMKKNNGNFVTDNFVKKSWEISNGVLIPPRRKNLFLYIDPSKICEYKKDPKLFKDFIYWSAFTEVERLKKAYGGARAKVVHAMKYSFTDIGSIIKGDDMMEKNSSDKIGKILGDTDGQNEKRKKWWDMNKYHIWESMLCGYREAEGDTETNENCRFPDIESVPQFLRWFQEWSENFCDRRQKLYDKLNSECISAECTNGSVDNSKCTHACVNYKNYILTKKTEYEIQTNKYDNEFKNKNSNDKDAPDYLKEKCNDNKCECLNKHIDDKNKTWKNPYETLEDTFKSKCDCPKP'

In [11]:
# Writing to file (test)
# Remember sequences have to end with the * character
with open("protein-alignment.ali", "w") as file:
    # Writing structure section
    file.write(">P1;" + structure.id + "\n")
    file.write("structureX:" + structure.id + 8*':' + "\n")
    for line in textwrap.wrap(cadena+"*", width=75, break_on_hyphens=False):
        file.write("%s\n" % line)
    # Writing sequence section
    file.write(">P1;" + structure.id + "_fill\n")
    file.write("sequence:" + 9*':' + "\n")
    for line in textwrap.wrap(full_seq+"*", width=75, break_on_hyphens=False):
        file.write("%s\n" % line)

# Completing the residues with modeller

based on https://salilab.org/modeller/wiki/Missing%20residues and http://www.msg.ucsf.edu/local/programs/modeller/node380.html#alignmentformat (for the alignment file format)

In [13]:
from modeller import *
from modeller.automodel import *    # Load the automodel class

log.verbose()
env = environ()

# directories for input atom files
env.io.atom_files_directory = ['.', 'atom_files']

a = loopmodel(env, alnfile = 'protein-alignment.ali',
              knowns = 'dbl6e', sequence = 'dbl6e_fill')
a.starting_model= 1
a.ending_model  = 1

a.loop.starting_model = 1
a.loop.ending_model   = 2
a.loop.md_level       = refine.fast

a.make()


openf___224_> Open           $(LIB)/restyp.lib
openf___224_> Open           ${MODINSTALL9v21}/modlib/resgrp.lib
rdresgr_266_> Number of residue groups:        2
openf___224_> Open           ${MODINSTALL9v21}/modlib/sstruc.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       455816     445.133     0.435

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       456344     445.648     0.435
openf___224_> Open           ${MODINSTALL9v21}/modlib/resdih.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       504944     493.109     0.482
rdrdih__263_> Number of dihedral angle types         :        9
              Maximal number of dihedral angle optima:        3
              Dihedral angle names                   :  Alph Phi Psi Omeg chi1 chi2 chi3 chi4 chi5
openf___224_> Open           ${MODINSTALL9v21}/modlib/radii.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       518244     506.098     0.494
openf___224_> Open           $

OSError: pdbnam_____E> Filename for PDB code not found: dbl6e
              Directories: .:atom_files
              Extensions : :.atm:.pdb:.ent:.cif:.crd
              (Also tried prepending 'pdb', looking for .Z, .gz, .bz2, .7z, .xz,
              and trying PDB-style subdirectories - e.g. ab for pdb1abc.ent)


# Sandbox

In [ ]:
cadena.replace("-", "wtf?!", 1)

In [ ]:
type(seq1(mis_res['res_name']))

In [ ]:
structure.id

In [ ]:
a = "hola"

In [ ]:
a.join(structure.id)

In [ ]:
7*':'